In [ ]:
import pandas as pd
import os
import numpy as np
from datetime import datetime, timedelta
import datetime as dt
from pylab import mpl, plt
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

In [ ]:
# Create a list of all year-month combinations from Jan 2015 to Aug 2025
date_range = pd.date_range(start='2015-01-01', end='2025-08-01', freq='MS')

# Generate filenames based on the format
filenames = [f"FMTQIK_{date.year}{date.month:02d}.csv" for date in date_range]

header=1
footer=2
# Read and concatenate all CSVs into a single DataFrame
df = pd.concat([pd.read_csv(filename,skiprows=header, skipfooter=footer,engine='python') for filename in filenames], ignore_index=True)
df = df.dropna(axis=1, how='all')

In [ ]:
# Clean and convert relevant columns
columns_to_clean = ['Trade Volume', 'Trade Value', 'Transaction', 'TAIEX']
for col in columns_to_clean:
    df[col] = df[col].str.replace(',', '')
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Convert Date column to datetime
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

# Drop rows with NaN in Date, TAIEX, or Trade Volume
df_clean = df.dropna(subset=['Date', 'TAIEX', 'Trade Volume'])

In [ ]:
# Create line chart of TAIEX vs Trade Volume
fig = px.line(df_clean, x='Date', y=['TAIEX', 'Trade Volume'],
              labels={'value': 'Value', 'variable': 'Legend'},
              title='TAIEX Index vs Trade Volume Over Time')

fig.show()

In [ ]:
# Create line chart of TAIEX vs Trade Volume
fig = px.line(df_clean, x='Date', y=['TAIEX'],
              labels={'value': 'Value', 'variable': 'Legend'},
              title='TAIEX Index Over Time')
fig.show()

In [ ]:
# Create subplots with 70/30 height ratio
fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
                    row_heights=[0.7, 0.3],
                    subplot_titles=("TAIEX Index Over Time", "Trade Volume Over Time"))

# Add TAIEX line
fig.add_trace(go.Scatter(x=df_clean['Date'], y=df_clean['TAIEX'],
                         mode='lines', name='TAIEX'), row=1, col=1)

# Add Trade Volume line
fig.add_trace(go.Scatter(x=df_clean['Date'], y=df_clean['Trade Volume'],
                         mode='lines', name='Trade Volume'), row=2, col=1)

# Update layout
fig.update_layout(height=800, title_text="TAIEX Index and Trade Volume (70/30 Split)")

fig.show()


In [ ]:
# Create a list of all year-month combinations from Jan 2015 to Aug 2025
date_range = pd.date_range(start='2015-01-01', end='2025-08-01', freq='MS')

# Generate filenames based on the format
filenames = [f"FMTQIK_{date.year}{date.month:02d}.csv" for date in date_range]

header=1
footer=2
# Read and concatenate all CSVs into a single DataFrame
df = pd.concat([pd.read_csv(filename,skiprows=header, skipfooter=footer,engine='python') for filename in filenames], ignore_index=True)
df = df.dropna(axis=1, how='all')
#df = df.set_index('Date')
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

df = df.dropna()

# Drop rows with NaN in Date, TAIEX, or Trade Volume
df = df.dropna(subset=['Date', 'TAIEX', 'Trade Volume'])

columns_to_clean = ['Trade Volume', 'Trade Value', 'Transaction', 'TAIEX']
for col in columns_to_clean:
    df[col] = df[col].str.replace(',', '')
    df[col] = pd.to_numeric(df[col], errors='coerce')

df.to_csv('FMTQIK_2025-0903.csv')
# Optional: display the first few rows
print(df.head())